## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,Date,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,2021-01-08 05:34:58,Kapaa,US,22.0752,-159.3190,73.40,78,40,3.44,scattered clouds
1,1,2021-01-08 05:34:58,Rezina,MD,47.7492,28.9622,32.47,92,100,4.38,overcast clouds
2,2,2021-01-08 05:34:59,Jamestown,US,42.0970,-79.2353,25.00,93,90,5.75,overcast clouds
3,3,2021-01-08 05:34:59,Albany,US,42.6001,-73.9662,26.01,75,17,6.13,few clouds
4,4,2021-01-08 05:34:59,Pryazha,RU,61.6926,33.6216,15.80,92,90,4.47,light snow


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minumum temperature of your vacation destination?"))
max_temp = float(input("What is the maximum temperature of your vacation destination?"))

What is the minumum temperature of your vacation destination?75
What is the maximum temperature of your vacation destination?90


In [22]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & \
                                       (city_data_df["Max Temp"] <= max_temp)]
preferred_cities_df.head()

,City_ID,Date,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,2021-01-08 05:34:59,Vaini,TO,-21.2000,-175.2000,82.40,83,75,5.75,broken clouds
7,7,2021-01-08 05:35:00,Hobyo,SO,5.3505,48.5268,77.22,62,8,23.69,clear sky
16,16,2021-01-08 05:35:05,Lugoba,TZ,-6.4500,38.3333,78.57,75,76,7.25,broken clouds
23,23,2021-01-08 05:35:07,Georgetown,MY,5.4112,100.3354,88.00,62,20,6.91,few clouds
25,25,2021-01-08 05:35:07,Saint-Pierre,RE,-21.3393,55.4781,82.40,65,0,20.71,clear sky


In [23]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
Date                   0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [27]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
Date                   0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vaini,TO,82.40,broken clouds,-21.2000,-175.2000,
7,Hobyo,SO,77.22,clear sky,5.3505,48.5268,
16,Lugoba,TZ,78.57,broken clouds,-6.4500,38.3333,
23,Georgetown,MY,88.00,few clouds,5.4112,100.3354,
25,Saint-Pierre,RE,82.40,clear sky,-21.3393,55.4781,
30,Kavieng,PG,84.13,moderate rain,-2.5744,150.7967,
32,New Norfolk,AU,75.00,clear sky,-42.7826,147.0587,
38,Rikitea,PF,78.60,broken clouds,-23.1203,-134.9692,
41,Moerai,PF,78.75,scattered clouds,-22.4333,-151.3333,
64,Arraial Do Cabo,BR,75.51,broken clouds,-22.9661,-42.0278,


In [35]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except IndexError:
        print("Hotel not found. Skipping...")

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


In [68]:
# 7. Drop the rows where there is no Hotel Name.
no_value_hotels = hotel_df[hotel_df["Hotel Name"] == ""].index
hotel_df.drop(no_value_hotels, inplace=True) # advice gotten from: https://thispointer.com/python-pandas-how-to-drop-rows-in-dataframe-by-conditions-on-column-values/
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Vaini,TO,82.40,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
7,Hobyo,SO,77.22,clear sky,5.3505,48.5268,Halane Hotel and Restaurent
23,Georgetown,MY,88.00,few clouds,5.4112,100.3354,Cititel Penang
25,Saint-Pierre,RE,82.40,clear sky,-21.3393,55.4781,Lindsey Hôtel
30,Kavieng,PG,84.13,moderate rain,-2.5744,150.7967,Nusa Island Retreat
...,...,...,...,...,...,...,...
712,Pringsewu,ID,87.64,moderate rain,-5.3582,104.9744,Urban Style by Front One
714,Amahai,ID,85.19,scattered clouds,-3.3333,128.9167,HOTEL ISABELA
717,Mogadishu,SO,77.00,few clouds,2.0371,45.3438,Hotel Juba
718,Beira,MZ,82.40,moderate rain,-19.8436,34.8389,Beira Terrace Hotel


In [64]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [69]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>city name</dt><dd>{City}</dd>
<dt>country</dt><dd>{Country}</dd>
<dt>weather description</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [70]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))